# Full audio

## Feature

### Set up

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COSWARA - HYBRID MODEL FULL AUDIO')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [3]:
n_mfcc = 26

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet'

meta_coswara = os.path.join(root, 'full_audio/coswara_metadata_for_label_matching.csv')
meta_coughvid = os.path.join(root, 'full_audio/coughvid_metadata_for_label_matching.csv')


mfcc_coswara = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coswara_features.npy')
mfcc_coughvid_1 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_20k.npy')
mfcc_coughvid_2 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_20k_40k.npy')
mfcc_coughvid_3 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_40k_60k.npy')
mfcc_coughvid_4 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_60k_75k.npy')
mfcc_coughvid_5 = os.path.join(root, f'full_audio/mfcc_{n_mfcc}_coughvid_features_9981.npy')


### Feature preparation

#### Metadata

In [4]:
# process metadata
meta_df = pd.read_csv(meta_coswara)
meta_df.head()

,id,age,covid_status,record_date,is_english_proficiency,gender,country,locality,state,is_returning_user,...,is_fatigue,is_sore_throat,is_ischemic_heart_disease,is_asthma,is_others_preexist_conditions,is_chronic_lung_disease,is_neumonia,cough_path,label,error
0,iV3Db6t1T8b7c5HQY2TwxIhjbzD3,28,healthy,2020-04-23,y,male,India,Anantapur,Andhra Pradesh,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,healthy,0
1,AxuYWBN0jFVLINCBqIW5aZmGCdu1,25,healthy,2020-04-20,y,male,India,BENGALURU URBAN,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/AxuYWBN0jFVLINCBqIW5aZmGCdu1/cough-shallow.wav,healthy,0
2,C5eIsssb9GSkaAgIfsHMHeR6fSh1,28,healthy,2020-04-24,y,female,United States,Pittsburgh,Pennsylvania,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/C5eIsssb9GSkaAgIfsHMHeR6fSh1/cough-shallow.wav,healthy,0
3,YjbEAECMBIaZKyfqOvWy5DDImUb2,26,healthy,2020-04-23,y,male,India,Bangalore,Karnataka,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/YjbEAECMBIaZKyfqOvWy5DDImUb2/cough-shallow.wav,healthy,0
4,aGOvk4ji0cVqIzCs1jHnzlw2UEy2,32,healthy,2020-04-22,y,male,India,Nalanda,Bihar,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coswara/coswara_cough_data/20200424/aGOvk4ji0cVqIzCs1jHnzlw2UEy2/cough-shallow.wav,healthy,0


In [5]:
logger.info(meta_df.shape)

2022-04-28 16:12:09,911 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - (4465, 39)


In [6]:
mfcc_feature = np.load(mfcc_coswara)
meta_df = pd.read_csv(meta_coswara)
meta_df = meta_df[meta_df['error'] == 0].reset_index(drop=True)

In [7]:
logger.info(meta_df.shape)

2022-04-28 16:12:11,034 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - (4400, 39)


In [8]:
meta_df.label.value_counts().sum()

4400

In [9]:
mfcc_feature[0].shape, len(mfcc_feature), meta_df.shape

((234, 246), 4400, (4400, 39))

In [10]:
DROP_COLS = [
             'cough_path',
             'error',
             'record_date',
             'id',
             'covid_status',
             'test_date'
]

meta_df.isna().sum()

id                                  0
age                                 0
covid_status                        0
record_date                         0
is_english_proficiency              0
gender                              0
country                             0
locality                          471
state                               0
is_returning_user                1335
is_smoker                        3273
is_cold                          3825
is_hypertension                  4042
is_diabetes                      4024
is_cough                         3638
date_of_ct_scan                  4135
has_ctScan                       3510
ct_score                         4135
is_diarrheoa                     4332
is_fever                         3965
is_loss_of_smell                 4153
is_muscle_pain                   4031
test_type                        3657
test_date                        3657
test_status                      2772
is_using_mask                    2040
vaccination_

In [11]:
# fill na with unknown value
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

id                               0
age                              0
covid_status                     0
record_date                      0
is_english_proficiency           0
gender                           0
country                          0
locality                         0
state                            0
is_returning_user                0
is_smoker                        0
is_cold                          0
is_hypertension                  0
is_diabetes                      0
is_cough                         0
date_of_ct_scan                  0
has_ctScan                       0
ct_score                         0
is_diarrheoa                     0
is_fever                         0
is_loss_of_smell                 0
is_muscle_pain                   0
test_type                        0
test_date                        0
test_status                      0
is_using_mask                    0
vaccination_status               0
is_breathing_difficulty          0
is_others_resp      

In [12]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)
logger.info(meta_df.shape)

2022-04-28 16:12:11,111 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - (4400, 33)


In [13]:
# replace label with binary numbers
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'positive_mild': 1,
    'no_resp_illness_exposed': 1,
    'resp_illness_not_identified': 0,
    'positive_moderate': 1,
    'recovered_full': 0,
    'positive_asymp': 1
})
meta_df.label.value_counts()

0    3194
1    1206
Name: label, dtype: int64

In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label', 'age'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [15]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

In [16]:
mfcc_feature = np.load(mfcc_coswara)

##### 2d

In [17]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [18]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-04-28 16:12:12,526 - COSWARA - HYBRID MODEL FULL AUDIO - INFO - 1206, 3194


In [19]:
# get features for each class
feature_0 = mfcc_feature[index_0]
feature_1 = mfcc_feature[index_1]

In [20]:
print(len(feature_1))

1206


In [21]:
del mfcc_feature

In [22]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:2555], feature_1[:965]), axis=0)
  val = np.concatenate((feature_0[2555:2875], feature_1[965:1086]), axis=0)
  test = np.concatenate((feature_0[2875:], feature_1[1086:]), axis=0)

  y_train = pd.concat([label0[:2555], label1[:965]], ignore_index=True)
  y_val = pd.concat([label0[2555:2875], label1[965:1086]], ignore_index=True)
  y_test = pd.concat([label0[2875:], label1[1086:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [23]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(3520, 234, 246)


In [24]:
y_train_2d

array([0, 0, 0, ..., 1, 1, 1])

In [25]:
del feature_0
del feature_1

##### 1d

In [26]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [27]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [28]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d, random_state=42)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d, random_state=42)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d, random_state=42)

## Model

In [30]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(3520, 234, 246, 1)

In [31]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(3520, 32, 1)

In [32]:
X_train_1d.shape

(3520, 32, 1)

In [33]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [34]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [35]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [36]:
# from tensorflow.keras.utils import plot_model
# plot_model(merged_model, to_file='model_final.png')
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [37]:
from datetime import datetime
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [38]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

Epoch 1/40
440/440 [==============================] - 49s 70ms/step - loss: 0.6677 - auc: 0.7163 - val_loss: 0.6499 - val_auc: 0.6671
Epoch 2/40
440/440 [==============================] - 30s 68ms/step - loss: 0.6124 - auc: 0.7365 - val_loss: 0.6060 - val_auc: 0.7287
Epoch 3/40
440/440 [==============================] - 29s 67ms/step - loss: 0.5949 - auc: 0.7488 - val_loss: 0.5869 - val_auc: 0.7972
Epoch 4/40
440/440 [==============================] - 29s 65ms/step - loss: 0.5877 - auc: 0.7528 - val_loss: 0.5768 - val_auc: 0.7702
Epoch 5/40
440/440 [==============================] - 29s 66ms/step - loss: 0.5806 - auc: 0.7621 - val_loss: 0.5948 - val_auc: 0.7370
Epoch 6/40
440/440 [==============================] - 30s 67ms/step - loss: 0.5719 - auc: 0.7724 - val_loss: 0.5412 - val_auc: 0.8097
Epoch 7/40
440/440 [==============================] - 29s 66ms/step - loss: 0.5711 - auc: 0.7769 - val_loss: 0.5966 - val_auc: 0.7382
Epoch 8/40
440/440 [==============================] - 29s 66ms

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_26_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_26_hybrid_model_40_8_0.001/assets


Epoch 1/40
220/220 [==============================] - 28s 113ms/step - loss: 0.5401 - auc: 0.8048 - val_loss: 0.5578 - val_auc: 0.7945
Epoch 2/40
220/220 [==============================] - 24s 111ms/step - loss: 0.5271 - auc: 0.8165 - val_loss: 0.6808 - val_auc: 0.6102
Epoch 3/40
220/220 [==============================] - 24s 111ms/step - loss: 0.5322 - auc: 0.8130 - val_loss: 0.6040 - val_auc: 0.7894
Epoch 4/40
220/220 [==============================] - 24s 111ms/step - loss: 0.5267 - auc: 0.8147 - val_loss: 0.5557 - val_auc: 0.7936
Epoch 5/40
220/220 [==============================] - 24s 111ms/step - loss: 0.5173 - auc: 0.8235 - val_loss: 0.5590 - val_auc: 0.7882
Epoch 6/40
14/14 [==============================] - 0s 28ms/step - loss: 0.6316 - auc: 0.6866


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_26_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_26_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_26_hybrid_model_40_64_0.001/assets


Epoch 1/40
28/28 [==============================] - 22s 740ms/step - loss: 0.0852 - auc: 0.9939 - val_loss: 1.5681 - val_auc: 0.7274
Epoch 2/40
28/28 [==============================] - 21s 737ms/step - loss: 0.0370 - auc: 0.9980 - val_loss: 2.1397 - val_auc: 0.6791
Epoch 3/40
28/28 [==============================] - 21s 744ms/step - loss: 0.0265 - auc: 0.9991 - val_loss: 2.1607 - val_auc: 0.6922
Epoch 4/40
28/28 [==============================] - 20s 735ms/step - loss: 0.0247 - auc: 0.9988 - val_loss: 1.9085 - val_auc: 0.7252
Epoch 5/40
28/28 [==============================] - 21s 738ms/step - loss: 0.0349 - auc: 0.9972 - val_loss: 2.0009 - val_auc: 0.7064
Epoch 6/40
14/14 [==============================] - 0s 28ms/step - loss: 1.9374 - auc: 0.7317


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_26_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coswara_mfcc_26_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [29]:
X_train_2d.shape

(3520, 234, 246)

In [30]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(3520, 234, 246, 1)

In [31]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(3520, 32, 1)

In [32]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [33]:
y_train_2d.shape

TensorShape([3520, 2])

In [34]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [35]:
LEARNING_RATE = 0.001
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5


opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [37]:
fp = os.path.join(SAVED_MODEL_PATH, '/coswara_melme_hybrid_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [38]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

Epoch 1/40
440/440 [==============================] - 39s 69ms/step - loss: 0.6609 - auc: 0.7228 - val_loss: 0.5960 - val_auc: 0.7317
Epoch 2/40
440/440 [==============================] - 29s 66ms/step - loss: 0.5965 - auc: 0.7527 - val_loss: 0.5769 - val_auc: 0.7702
Epoch 3/40
440/440 [==============================] - 29s 66ms/step - loss: 0.5832 - auc: 0.7641 - val_loss: 0.6471 - val_auc: 0.7800
Epoch 4/40
440/440 [==============================] - 29s 66ms/step - loss: 0.5902 - auc: 0.7546 - val_loss: 0.5488 - val_auc: 0.8064
Epoch 5/40
440/440 [==============================] - 28s 64ms/step - loss: 0.5824 - auc: 0.7625 - val_loss: 0.5793 - val_auc: 0.7908
Epoch 6/40
440/440 [==============================] - 28s 65ms/step - loss: 0.5700 - auc: 0.7767 - val_loss: 0.6199 - val_auc: 0.7555
Epoch 7/40
440/440 [==============================] - 28s 65ms/step - loss: 0.5619 - auc: 0.7860 - val_loss: 0.6187 - val_auc: 0.7653
Epoch 8/40
440/440 [==============================] - 29s 66ms

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mfcc_26_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mfcc_26_hybrid_model_40_8_0.001/assets


Epoch 1/40
220/220 [==============================] - 29s 115ms/step - loss: 0.5444 - auc: 0.8033 - val_loss: 0.5338 - val_auc: 0.8112
Epoch 2/40
220/220 [==============================] - 25s 111ms/step - loss: 0.5327 - auc: 0.8118 - val_loss: 0.5467 - val_auc: 0.8087
Epoch 3/40
220/220 [==============================] - 25s 112ms/step - loss: 0.5270 - auc: 0.8168 - val_loss: 0.5682 - val_auc: 0.7807
Epoch 4/40
220/220 [==============================] - 25s 112ms/step - loss: 0.5228 - auc: 0.8211 - val_loss: 0.5981 - val_auc: 0.7450
Epoch 5/40
220/220 [==============================] - 24s 111ms/step - loss: 0.5136 - auc: 0.8267 - val_loss: 0.6593 - val_auc: 0.7653
Epoch 6/40
14/14 [==============================] - 0s 28ms/step - loss: 3.7597 - auc: 0.3327


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mfcc_26_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mfcc_26_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mfcc_26_hybrid_model_40_64_0.001/assets


Epoch 1/40
28/28 [==============================] - 22s 743ms/step - loss: 0.0285 - auc: 0.9988 - val_loss: 2.2206 - val_auc: 0.7497
Epoch 2/40
28/28 [==============================] - 20s 728ms/step - loss: 0.0166 - auc: 0.9998 - val_loss: 2.2076 - val_auc: 0.7474
Epoch 3/40
28/28 [==============================] - 20s 726ms/step - loss: 0.0153 - auc: 0.9998 - val_loss: 2.6294 - val_auc: 0.7480
Epoch 4/40
28/28 [==============================] - 20s 733ms/step - loss: 0.0142 - auc: 0.9999 - val_loss: 2.4599 - val_auc: 0.7634
Epoch 5/40
28/28 [==============================] - 21s 734ms/step - loss: 0.0147 - auc: 0.9999 - val_loss: 2.4597 - val_auc: 0.7184
Epoch 6/40
28/28 [==============================] - 20s 733ms/step - loss: 0.0136 - auc: 0.9999 - val_loss: 2.5353 - val_auc: 0.7302
Epoch 7/40
28/28 [==============================] - 20s 721ms/step - loss: 0.0134 - auc: 0.9999 - val_loss: 2.6472 - val_auc: 0.7223
Epoch 8/40
28/28 [==============================] - 21s 736ms/step - 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mfcc_26_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coswara_mfcc_26_hybrid_model_40_128_0.001/assets


## SMOTE

In [29]:
X_train_2d.shape

(3520, 234, 246)

In [30]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(5110, 234, 246, 1)

In [31]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(5110, 32, 1)

In [32]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [33]:
y_train_2d.shape

TensorShape([5110, 2])

In [34]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [35]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [36]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [38]:
fp = os.path.join(SAVED_MODEL_PATH, '/coswara_hybrid_mel_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [39]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

Epoch 1/40
639/639 [==============================] - 49s 65ms/step - loss: 0.6720 - auc: 0.6978 - val_loss: 1.0750 - val_auc: 0.2356
Epoch 2/40
639/639 [==============================] - 40s 62ms/step - loss: 0.5828 - auc: 0.7645 - val_loss: 1.7977 - val_auc: 0.7350
Epoch 3/40
639/639 [==============================] - 40s 63ms/step - loss: 0.5538 - auc: 0.7943 - val_loss: 0.7862 - val_auc: 0.7606
Epoch 4/40
639/639 [==============================] - 40s 63ms/step - loss: 0.5585 - auc: 0.7901 - val_loss: 0.5786 - val_auc: 0.7728
Epoch 5/40
639/639 [==============================] - 40s 62ms/step - loss: 0.5286 - auc: 0.8180 - val_loss: 0.6979 - val_auc: 0.7371
Epoch 6/40
639/639 [==============================] - 39s 62ms/step - loss: 0.5218 - auc: 0.8198 - val_loss: 0.6053 - val_auc: 0.7541
Epoch 7/40
639/639 [==============================] - 39s 62ms/step - loss: 0.5040 - auc: 0.8335 - val_loss: 2.1932 - val_auc: 0.3294
Epoch 8/40
639/639 [==============================] - 39s 62ms

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_26_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_26_hybrid_model_40_8_0.001/assets


Epoch 1/40
320/320 [==============================] - 34s 107ms/step - loss: 0.4116 - auc: 0.8942 - val_loss: 0.6560 - val_auc: 0.6864
Epoch 2/40
320/320 [==============================] - 34s 106ms/step - loss: 0.3768 - auc: 0.9107 - val_loss: 0.8672 - val_auc: 0.6363
Epoch 3/40
320/320 [==============================] - 34s 106ms/step - loss: 0.3620 - auc: 0.9170 - val_loss: 0.7236 - val_auc: 0.7240
Epoch 4/40
320/320 [==============================] - 34s 106ms/step - loss: 0.3251 - auc: 0.9329 - val_loss: 0.9964 - val_auc: 0.5999
Epoch 5/40
320/320 [==============================] - 34s 106ms/step - loss: 0.2933 - auc: 0.9440 - val_loss: 0.7648 - val_auc: 0.7498
Epoch 6/40
320/320 [==============================] - 34s 106ms/step - loss: 0.2665 - auc: 0.9537 - val_loss: 1.0779 - val_auc: 0.7339
Epoch 7/40
320/320 [==============================] - 34s 106ms/step - loss: 0.2278 - auc: 0.9662 - val_loss: 1.0280 - val_auc: 0.7291
Epoch 8/40
320/320 [==============================] - 3

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_26_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_26_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_26_hybrid_model_40_64_0.001/assets


Epoch 1/40
40/40 [==============================] - 31s 746ms/step - loss: 0.0190 - auc: 0.9994 - val_loss: 2.8953 - val_auc: 0.6990
Epoch 2/40
40/40 [==============================] - 29s 718ms/step - loss: 0.0132 - auc: 0.9999 - val_loss: 2.9615 - val_auc: 0.6934
Epoch 3/40
40/40 [==============================] - 29s 714ms/step - loss: 0.0140 - auc: 0.9999 - val_loss: 3.0847 - val_auc: 0.6857
Epoch 4/40
40/40 [==============================] - 29s 720ms/step - loss: 0.0134 - auc: 0.9999 - val_loss: 3.0948 - val_auc: 0.6984
Epoch 5/40
40/40 [==============================] - 29s 734ms/step - loss: 0.0136 - auc: 0.9999 - val_loss: 3.0787 - val_auc: 0.6978
Epoch 6/40
14/14 [==============================] - 0s 28ms/step - loss: 2.9821 - auc: 0.7260


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_26_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coswara_mfcc_26_hybrid_model_40_128_0.001/assets


## Evaluation

In [29]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [30]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [31]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [32]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 3s 36ms/step - loss: 2.5715 - auc: 0.7252


[2.5715153217315674, 0.7252194881439209]

In [33]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 1s 29ms/step - loss: 1.7505 - auc: 0.7532


[1.750527024269104, 0.7531872987747192]

In [34]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 0.7429 - auc: 0.7894


[0.7428513169288635, 0.7893691062927246]

In [35]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 1s 29ms/step - loss: 3.7597 - auc: 0.3327


[3.759674310684204, 0.33274006843566895]

In [36]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 0.5449 - auc: 0.8319


[0.5448674559593201, 0.8318657875061035]

### Raw features

In [37]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [38]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 1.9374 - auc: 0.7317


[1.9374207258224487, 0.7316587567329407]

In [39]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 29ms/step - loss: 2.5867 - auc: 0.5180


[2.5867323875427246, 0.5180078744888306]

In [40]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 29ms/step - loss: 0.5797 - auc: 0.7976


[0.5796558260917664, 0.7975596785545349]

In [41]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 29ms/step - loss: 0.6316 - auc: 0.6866


[0.6316385269165039, 0.6865754723548889]

In [42]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 0.5317 - auc: 0.8191


[0.5317367911338806, 0.8190701007843018]

### SMOTE

In [43]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 2.9821 - auc: 0.7260


[2.9820621013641357, 0.7259770035743713]

In [44]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 2.9599 - auc: 0.7275


[2.9598946571350098, 0.7275128364562988]

In [45]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 30ms/step - loss: 1.5105 - auc: 0.7906


[1.5104538202285767, 0.7905936241149902]

In [46]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 28ms/step - loss: 1.0643 - auc: 0.7613


[1.0642526149749756, 0.7612818479537964]

In [47]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_full_audio_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

14/14 [==============================] - 1s 29ms/step - loss: 0.5745 - auc: 0.7949


[0.5745381712913513, 0.7949004173278809]

# Chunks
* potentials: use `num_chunk` to explore as long audio influence the model'accuracy (the more `num_chunk` the longer the audio)

## Feature

### Set up

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COSWARA - HYBRID MODEL CHUNKS')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [ ]:
n_mfcc = 26

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet'
data_root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing'

meta_coswara = os.path.join(data_root, 'cough_chunking/coswara/coswara_chunk_metadata.csv')
meta_coughvid = os.path.join(data_root, 'cough_chunking/coughvid/coughvid_chunk_metadata.csv')

mel_coswara = os.path.join(root, 'chunks/melspectrogram_coswara_features.npy')
mel_coughvid_1 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_20k.npy')
mel_coughvid_2 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_20k_40k.npy')
mel_coughvid_3 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_40k_60k.npy')
mel_coughvid_4 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_60k_75k.npy')
mel_coughvid_5 = os.path.join(root, 'chunks/melspectrogram_coughvid_features_9981.npy')

mfcc_coswara = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coswara_features.npy')
mfcc_coughvid_1 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_20k.npy')
mfcc_coughvid_2 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_20k_40k.npy')
mfcc_coughvid_3 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_40k_60k.npy')
mfcc_coughvid_4 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_60k_75k.npy')
mfcc_coughvid_5 = os.path.join(root, f'chunks/mfcc_{n_mfcc}_coughvid_features_9981.npy')


### Feature preparation

#### Metadata

In [ ]:
# process metadata
meta_df = pd.read_csv(meta_coswara)
meta_df.head()

,uuid,age,record_date,is_english_proficiency,gender,country,locality,state,is_returning_user,is_smoker,...,is_sore_throat,is_ischemic_heart_disease,is_asthma,is_others_preexist_conditions,is_chronic_lung_disease,is_neumonia,label,sr_cough,chunk_path,num_chunk
0,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-0,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-0.wav,8
1,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-1,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-1.wav,8
2,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-2,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-2.wav,8
3,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-3,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-3.wav,8
4,20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-4,28,2020-04-23,y,male,india,anantapur,andhra pradesh,n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coswara_cough_data/20200424/iV3Db6t1T8b7c5HQY2TwxIhjbzD3/cough-shallow.wav,coswara_cough_chunk/20200424-iV3Db6t1T8b7c5HQY2TwxIhjbzD3-cough-shallow-4.wav,8


In [ ]:
logger.info(meta_df.shape)

2022-04-27 09:36:24,848 - COSWARA - HYBRID MODEL CHUNKS - INFO - (22878, 39)


In [ ]:
DROP_COLS = [
             'sr_cough',
             'chunk_path',
             'record_date',
             'uuid',
             'test_date'
]

meta_df.isna().sum()

uuid                                 0
age                                  0
record_date                          0
is_english_proficiency               0
gender                               0
country                              0
locality                          2295
state                                0
is_returning_user                 7281
is_smoker                        17592
is_cold                          20117
is_hypertension                  21030
is_diabetes                      20985
is_cough                         19040
date_of_ct_scan                  21754
has_ctScan                       18880
ct_score                         21754
is_diarrheoa                     22514
is_fever                         20704
is_loss_of_smell                 21621
is_muscle_pain                   21229
test_type                        19560
test_date                        19560
test_status                      14906
is_using_mask                    11268
vaccination_status       

In [ ]:
# fill na with unknown value
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

uuid                             0
age                              0
record_date                      0
is_english_proficiency           0
gender                           0
country                          0
locality                         0
state                            0
is_returning_user                0
is_smoker                        0
is_cold                          0
is_hypertension                  0
is_diabetes                      0
is_cough                         0
date_of_ct_scan                  0
has_ctScan                       0
ct_score                         0
is_diarrheoa                     0
is_fever                         0
is_loss_of_smell                 0
is_muscle_pain                   0
test_type                        0
test_date                        0
test_status                      0
is_using_mask                    0
vaccination_status               0
is_breathing_difficulty          0
is_others_resp                   0
is_fatigue          

In [ ]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)
logger.info(meta_df.shape)

2022-04-27 09:36:24,994 - COSWARA - HYBRID MODEL CHUNKS - INFO - (22878, 34)


In [ ]:
# replace label with binary numbers
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'positive_mild': 1,
    'no_resp_illness_exposed': 1,
    'resp_illness_not_identified': 0,
    'positive_moderate': 1,
    'recovered_full': 0,
    'positive_asymp': 1
})
meta_df.label.value_counts()

0    16871
1     6007
Name: label, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label', 'age'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [ ]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

In [ ]:
mfcc_feature = np.load(mfcc_coswara)

##### 2d

In [ ]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [ ]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-04-27 09:36:42,321 - COSWARA - HYBRID MODEL CHUNKS - INFO - 6007, 16871


In [ ]:
# get features for each class
feature_0 = mfcc_feature[index_0]
feature_1 = mfcc_feature[index_1]

In [ ]:
print(len(feature_1))

6007


In [ ]:
del mfcc_feature

In [ ]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:13504], feature_1[:4803]), axis=0)
  val = np.concatenate((feature_0[13504:15186], feature_1[4803:5408]), axis=0)
  test = np.concatenate((feature_0[15186:], feature_1[5408:]), axis=0)

  y_train = pd.concat([label0[:13504], label1[:4803]], ignore_index=True)
  y_val = pd.concat([label0[13504:15186], label1[4803:5408]], ignore_index=True)
  y_test = pd.concat([label0[15186:], label1[5408:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [ ]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(18307, 234, 236)


In [ ]:
y_train_2d

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
del feature_0
del feature_1

##### 1d

In [ ]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [ ]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [ ]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d)

## Model

In [ ]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(18307, 234, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(18307, 33, 1)

In [ ]:
X_train_1d.shape

(18307, 33, 1)

In [ ]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(merged_model, to_file='model_final.png')
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)

Epoch 1/40
144/144 [==============================] - 122s 828ms/step - loss: 0.0130 - auc: 0.9996 - val_loss: 1.4752 - val_auc: 0.8060
Epoch 2/40
144/144 [==============================] - 121s 837ms/step - loss: 0.0053 - auc: 0.9998 - val_loss: 1.7569 - val_auc: 0.7985
Epoch 3/40
144/144 [==============================] - 122s 851ms/step - loss: 0.0045 - auc: 0.9998 - val_loss: 1.9980 - val_auc: 0.7876
Epoch 4/40
144/144 [==============================] - 121s 844ms/step - loss: 0.0422 - auc: 0.9963 - val_loss: 1.3550 - val_auc: 0.8034
Epoch 5/40
144/144 [==============================] - 122s 845ms/step - loss: 0.0531 - auc: 0.9955 - val_loss: 1.2883 - val_auc: 0.7951
Epoch 6/40
144/144 [==============================] - 122s 846ms/step - loss: 0.0095 - auc: 0.9996 - val_loss: 1.6122 - val_auc: 0.8094
Epoch 7/40
144/144 [==============================] - 122s 847ms/step - loss: 0.0050 - auc: 0.9999 - val_loss: 1.7466 - val_auc: 0.7879
Epoch 8/40
144/144 [============================

In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

72/72 [==============================] - 3s 37ms/step - loss: 1.5690 - auc: 0.7352


[1.568955659866333, 0.7352277040481567]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coswara_mfcc_26_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/coswara_mfcc_26_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [ ]:
X_train_2d.shape

(18307, 234, 236)

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(18307, 234, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(18307, 33, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([18307, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=256, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 256, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = os.path.join(SAVED_MODEL_PATH, f'/coswara_mfcc_{n_mfcc}_hybrid_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)

Epoch 1/40
144/144 [==============================] - 97s 670ms/step - loss: 0.0998 - auc: 0.9907 - val_loss: 1.8554 - val_auc: 0.6716
Epoch 2/40
144/144 [==============================] - 97s 676ms/step - loss: 0.0390 - auc: 0.9965 - val_loss: 2.0241 - val_auc: 0.6764
Epoch 3/40
144/144 [==============================] - 96s 669ms/step - loss: 0.0490 - auc: 0.9959 - val_loss: 2.4067 - val_auc: 0.7387
Epoch 4/40
144/144 [==============================] - 97s 672ms/step - loss: 0.0706 - auc: 0.9946 - val_loss: 2.3327 - val_auc: 0.7574
Epoch 5/40
144/144 [==============================] - 97s 671ms/step - loss: 0.0741 - auc: 0.9943 - val_loss: 2.3757 - val_auc: 0.7555
Epoch 6/40
144/144 [==============================] - 96s 669ms/step - loss: 0.1750 - auc: 0.9771 - val_loss: 1.6074 - val_auc: 0.7627
Epoch 7/40
144/144 [==============================] - 96s 669ms/step - loss: 0.0504 - auc: 0.9962 - val_loss: 1.7747 - val_auc: 0.7044
Epoch 8/40
144/144 [==============================] - 9

In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

72/72 [==============================] - 2s 27ms/step - loss: 1.3339 - auc: 0.7452


[1.3338708877563477, 0.7452110052108765]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'std_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coswara_mfcc_26_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_coswara_mfcc_26_hybrid_model_40_128_0.001/assets


## SMOTE

In [ ]:
X_train_2d.shape

(18307, 234, 236)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(27008, 234, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(27008, 33, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([27008, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
BATCH_SIZE = 128
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=256, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 256, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = os.path.join(SAVED_MODEL_PATH, f'/coswara_hybrid_mfcc_{n_mfcc}_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit([X_train_2d, X_train_1d],\
                    [y_train_2d, y_train_1d],\
                    epochs=EPOCHS,\
                    batch_size=BATCH_SIZE,\
                    validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                    callbacks=[check, earlystop_callback, tensorboard_callback],\
                    shuffle=True)

Epoch 1/40
211/211 [==============================] - 162s 683ms/step - loss: 0.6691 - auc: 0.6758 - val_loss: 0.7280 - val_auc: 0.2972
Epoch 2/40
211/211 [==============================] - 141s 668ms/step - loss: 0.5485 - auc: 0.7955 - val_loss: 0.7055 - val_auc: 0.5125
Epoch 3/40
211/211 [==============================] - 140s 665ms/step - loss: 0.5068 - auc: 0.8302 - val_loss: 8.3816 - val_auc: 0.2647
Epoch 4/40
211/211 [==============================] - 142s 671ms/step - loss: 0.4551 - auc: 0.8666 - val_loss: 2.6087 - val_auc: 0.7363
Epoch 5/40
211/211 [==============================] - 141s 671ms/step - loss: 0.4024 - auc: 0.8977 - val_loss: 1.1059 - val_auc: 0.7501
Epoch 6/40
211/211 [==============================] - 140s 666ms/step - loss: 0.3473 - auc: 0.9237 - val_loss: 4.3675 - val_auc: 0.7355
Epoch 7/40
211/211 [==============================] - 140s 666ms/step - loss: 0.2957 - auc: 0.9440 - val_loss: 2.5485 - val_auc: 0.7351
Epoch 8/40
211/211 [============================

In [ ]:
model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d], verbose=1)

72/72 [==============================] - 2s 28ms/step - loss: 2.3233 - auc: 0.7419


[2.323331356048584, 0.741946280002594]

In [ ]:
model.save(os.path.join(SAVED_MODEL_PATH, f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coswara_mfcc_26_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_coswara_mfcc_26_hybrid_model_40_128_0.001/assets


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs/fit

## Evaluation

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 16s 34ms/step - loss: 1.3339 - auc: 0.7452


[1.3338711261749268, 0.7452110052108765]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.9942 - auc: 0.7843


[0.9942086935043335, 0.7842556238174438]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.9282 - auc: 0.7837


[0.9281724095344543, 0.7836547493934631]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.7576 - auc: 0.8248


[0.7576499581336975, 0.8248071670532227]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'std_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.5688 - auc: 0.8132


[0.5688090324401855, 0.8132076263427734]

### Raw features

In [ ]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 1.5690 - auc: 0.7352


[1.568955898284912, 0.7352277040481567]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 1.4980 - auc: 0.7628


[1.4980002641677856, 0.7627788782119751]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 1.1758 - auc: 0.7578


[1.1757651567459106, 0.757781445980072]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.6615 - auc: 0.8209


[0.661478579044342, 0.820929765701294]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.5451 - auc: 0.8212


[0.5450502634048462, 0.8212292790412903]

### SMOTE

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 2.3233 - auc: 0.7419


[2.323331594467163, 0.741946280002594]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 30ms/step - loss: 1.9100 - auc: 0.7337


[1.9099724292755127, 0.7336890697479248]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 1.9399 - auc: 0.6844


[1.9399446249008179, 0.6843590140342712]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 3s 27ms/step - loss: 0.9632 - auc: 0.8083


[0.9632028341293335, 0.808282732963562]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       f'smote_coswara_mfcc_{n_mfcc}_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

72/72 [==============================] - 4s 27ms/step - loss: 0.7487 - auc: 0.6855


[0.7487303018569946, 0.685540497303009]